# k-NN model

Given a playlist, we want to add more tracks: it's the **playlist continuation** problem. Following [Kelen et al.](https://dl.acm.org/doi/abs/10.1145/3267471.3267477), the idea here is to define a similarity metric between two playlists, select the $k$ most similar playlists to ours, define a score metric for tracks continuing our playlist and choose the best tracks to continue it.

In [49]:
from scipy.sparse import lil_matrix
from tqdm.notebook import tqdm
import glob
import numpy as np
import pandas as pd

from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

## Treatment

### Tracks

Here we load and treat the tracks dataset.

In [50]:
tracks_dfs = (
    pd.read_pickle(file)[['id', 'playlist_id', 'artists_ids']] for file in glob.glob('../../data/old_data/sp_tracks_ready_*.pkl')
)
tracks_df = pd.concat(tracks_dfs, ignore_index=True)
tracks_df.dropna(inplace=True)
tracks_df

,id,playlist_id,artists_ids
0,0DABCqd96Fo71aZj6fKYcy,5sfxnGkrpCteZQt59RCGcL,"[2E4L9G4jTkc647FIq8rYGW, 11OUxHFoGgo2NDSdT6YiE..."
1,4jDF1QHbRoPUVFEkQn6Pit,5sfxnGkrpCteZQt59RCGcL,[4GaCmhB3S6i5A7y5vRJzsL]
2,0Qb9dMomcRHV8BLogGUqi2,5sfxnGkrpCteZQt59RCGcL,[5tIMc0MdfB2OV6sULOmeao]
3,3vt0qjq7IK1mBzR0PK45Oq,5sfxnGkrpCteZQt59RCGcL,[7dc6hUwyuIhrZdh80eaCEE]
4,4Id3igZYteLReHqHz8InL0,5sfxnGkrpCteZQt59RCGcL,[4IVlorYhuVoIUWg6HVx6DK]
...,...,...,...
849392,5Fw17rh1qEC6YoT32BijNI,1UggNlAE0AZL9jg5LgfnK6,[6ueGR6SWhUJfvEhqkvMsVs]
849393,29HemHbSIVerWQQ1YNVzXU,1UggNlAE0AZL9jg5LgfnK6,[6TqQLejnHXMGr7KcegxUND]
849394,0CPGEvqZ8wZAEYTQVcF2NL,1UggNlAE0AZL9jg5LgfnK6,[4lPl9gqgox3JDiaJ1yklKh]
849395,1Oc6sieJIKnw1Sy93Vp0Lq,1UggNlAE0AZL9jg5LgfnK6,"[0qM78DOdgnNPpq2CpTNgU5, 5BLDZvqf1kjdGL4jwFhAk5]"


In [51]:
playlists = tracks_df.groupby('playlist_id')['id'].apply(list)
playlists

playlist_id
002tR8Orc24qAjOUA1WZky    [4nRPcD4a5QGH95ZnwGvjwr, 66e6O0B9GmSRNAnzB0Bm7...
005DlsAeMYKNkiH68qg42D    [1EzZKyFDupKMTkanau4nBz, 0sCNal67P5Nydob9zITXr...
005TzWiU0IWtwpvJRl0E2U    [5sjINZsTACv5hmwv6nSrST, 2kH9pYAE74wdKG80AtL73...
00CgYEsYFmBBMY2xMc8ll2    [1Y4URywn643uZPth46W1Y4, 3iKlGM4UeSOecjb7pj10n...
00CxBYCkFSb8rXXI2HuEDw    [1uxzFavoQSzR6NhzeSbHdM, 7MoJIfitwutdf2B9RbJdd...
                                                ...                        
7ziIxFZYxrk5PGjmTIHw8U    [2IHaGyfxNoFPLJnaEg4GTs, 45e8T07myBjutIx2xh8Kq...
7zlLMQZc0JP6yStjLopDk9    [1MJ5f5EYBC92ADD6xcz7nb, 2J4obtL5oOmstcOfq5UW3...
7znebC1MKbXiQiUqxDGVMx    [2Fxmhks0bxGSBdJ92vM42m, 2qT1uLXPVPzGgFOx4jtEu...
7zr3klk3xuOVCKyxV4USDO    [6x0jeG72v8tYQfpfox6dWh, 1zcWL2FZKee3ub0wbTfNG...
7zsK5nJQzCjJaTnjiAcvd0    [3ZqZ3IbrVbFTdTvGcK0vpm, 1KqpIQGNINpG0hDdbB9Pi...
Name: id, Length: 10638, dtype: object

We treat the artists for each track:

In [52]:
artists = tracks_df.set_index('id').artists_ids
artists.index.name = 'track_id'
artists_ids = dict(zip(artists.index, range(len(artists))))
artists

track_id
0DABCqd96Fo71aZj6fKYcy    [2E4L9G4jTkc647FIq8rYGW, 11OUxHFoGgo2NDSdT6YiE...
4jDF1QHbRoPUVFEkQn6Pit                             [4GaCmhB3S6i5A7y5vRJzsL]
0Qb9dMomcRHV8BLogGUqi2                             [5tIMc0MdfB2OV6sULOmeao]
3vt0qjq7IK1mBzR0PK45Oq                             [7dc6hUwyuIhrZdh80eaCEE]
4Id3igZYteLReHqHz8InL0                             [4IVlorYhuVoIUWg6HVx6DK]
                                                ...                        
5Fw17rh1qEC6YoT32BijNI                             [6ueGR6SWhUJfvEhqkvMsVs]
29HemHbSIVerWQQ1YNVzXU                             [6TqQLejnHXMGr7KcegxUND]
0CPGEvqZ8wZAEYTQVcF2NL                             [4lPl9gqgox3JDiaJ1yklKh]
1Oc6sieJIKnw1Sy93Vp0Lq     [0qM78DOdgnNPpq2CpTNgU5, 5BLDZvqf1kjdGL4jwFhAk5]
4eFXaYD2is6c6Y645Qyyy1                             [3p3jPcp8b7WL9XYj4xlsWj]
Name: artists_ids, Length: 810282, dtype: object

## Training and test data

In [53]:
# Only playlists between 5 and 250 tracks
query = playlists.apply(lambda x: len(x))
playlists = playlists[(query >= 5) & (query <= 250)]

In [54]:
n_test = int(np.ceil(len(playlists)*0.2))
query = playlists.apply(lambda x: len(x))
playlists_test = playlists[(query > 25)].sample(n_test)

In [55]:
playlists_training = playlists.drop(index = playlists_test.index)

## The model

### Relevance matrix $R$

We build the relevance matrix $R$. $R_{ij}=r_{ji}$ indicates if a track $j$ is relevant to the playlist $i$, that is, the track is in the playlist.

In [56]:
all_tracks = []
for playlist in playlists_training.to_list():
    all_tracks.extend(playlist)
all_tracks = list(set(all_tracks))

Because we will use matrix multiplication, we have to index each track and each playlist id to index of the matrix. We do it here using dictionaries:

In [57]:
track_ids_go = dict(zip(all_tracks, range(len(all_tracks))))
track_ids_back = dict(zip(track_ids_go.values(), track_ids_go.keys()))
playlist_ids = dict(zip(set(playlists_training.index), range(len(set(playlists_training.index)))))

In [58]:
m = len(set(playlists_training.index))
n = len(set(all_tracks))
R = lil_matrix((m, n))

for playlist_id, playlist in tqdm(playlists_training.iteritems(), total=len(playlists_training)):
    for track_id in playlist:
        R[playlist_ids[playlist_id], track_ids_go[track_id]] = 1

### Similarity

The similarity between two playlists $u$ and $v$ is calculated by:
$$s_{uv} = \sum_{i \in I} \dfrac{r_{ui}r_{vi}}{||R_u||_2||R_v||_2}$$
$I$ is the set of tracks and $R_u$ is the vector of relevances $r_{ui}$ for the playlist $u$.

In fact, we basically count the number of tracks in the intersection of the playlists and normalize it.

In [59]:
def similarity(playlist_1, playlist_2):
    """Calculate the similarity between two playlists."""
    summation = len(set(playlist_1) & set(playlist_2))
    return summation/(np.sqrt(len(playlist_1))*np.sqrt(len(playlist_2)))

### Track score

Given a playlist $u$ to be continuated, we calculate the similarity of it with all existent playlists and select the $k$ most similar playlists, that is, the set $N_k(u)$. So, we define a score for a track to be in the playlist:
$$\hat{r}_{ui} = \dfrac{\sum_{v \in N_k(u)} s_{uv} \cdot r_{vi}}{\sum_{v \in N_k(u)} s_{uv}}$$

The intuition is that we are giving high scores to tracks that are in many playlists with great similarities to out playlist. We return the tracks ordered by score.

In [60]:
def continuation(playlist, k):
    """Continue a playlist based on k most similar playlists."""
    s_u = lil_matrix((1, m))
    for alt_playlist_index, alt_playlist in playlists_training.items():
        s_u[0, playlist_ids[alt_playlist_index]] = similarity(playlist, alt_playlist)
    sorted_similarities_indices = np.flip(np.argsort(s_u.toarray()[0]))
    top_k_similarities_indices = sorted_similarities_indices[:k]
    scores = (s_u[0, top_k_similarities_indices]*R[top_k_similarities_indices, :]).toarray()[0]
    sorted_scores_indices = np.flip(np.argsort(scores)[-500:])
    return [track_ids_back[index] for index in sorted_scores_indices]

## Evaluation

### R-precision metric

As described in it's work, [Chen et al.](https://dl.acm.org/doi/10.1145/3240323.3240342) suggest a metric for playlist continuation evaluation. They call it **R-precision**. It measures how many of the real tracks the model suggested correctly. In a variant, we will not consider the artists this time.

$$\textrm{R-precision} = \dfrac{||}{}$$

In [61]:
def r_precision(S_t, G_t, S_a, G_a):
    return (len(set(S_t) & set(G_t)) + 0.25 * len(set(S_a) & set(G_a))) / len(G_t)

In [62]:
def evaluation(playlist_not_hidden, playlist_hidden, continuation):
    continuation = continuation.copy()
    for track in set(playlist_not_hidden):
        if track in continuation:
            continuation.remove(track)
    continuation = continuation[:len(playlist_hidden)]
    
    G_a = []
    for track in playlist_hidden:
        G_a.extend(artists.iloc[artists_ids[track]])
    S_a = []
    for track in continuation:
        S_a.extend(artists.iloc[artists_ids[track]])
        
    metric = r_precision(continuation, playlist_hidden, S_a, G_a)
    return metric

In [97]:
k = len(playlists_training)

sum_metric = 0
sum_random = 0
for playlist in tqdm(playlists_test):
    playlist_not_hidden = playlist[:25]
    playlist_hidden = playlist[25:]
    continuated = continuation(playlist_not_hidden, k)
    metric = evaluation(playlist_not_hidden, playlist_hidden, continuated)
    random_metric = evaluation(playlist_not_hidden, playlist_hidden, random.sample(all_tracks, 500))
    sum_metric += metric
    sum_random += random_metric
print('The R-precision for our model is about {}.'.format(sum_metric/len(playlists_test)))
print('A random model performs {}.'.format(sum_random/len(playlists_test)))


The R-precision for our model is about 0.07761541803690636.
A random model performs 0.001648345006687441.


## Sanity check

Before evaluating out model, it's worth it to do a sanity check. We will take the most listened songs from The Beatles and continue the playlist, with $k = 100$.

In [119]:
q = sp.search('Pink Floyd', type='artist')

Our playlist to be continuated:

In [120]:
[track['name'] for track in sp.artist_top_tracks(q['artists']['items'][0]['id'])['tracks']]

['Wish You Were Here',
 'Another Brick in the Wall, Pt. 2',
 'Comfortably Numb',
 'Money',
 'Time',
 'Breathe (In the Air)',
 'Hey You',
 'Shine On You Crazy Diamond (Pts. 1-5)',
 'The Great Gig in the Sky',
 'Brain Damage']

Continuating...

In [121]:
temp = sp.artist_top_tracks(q['artists']['items'][0]['id'])['tracks']
result = continuation([track['id'] for track in temp], 1000)

So:

In [122]:
q = sp.tracks(result[:50])

In [123]:
[(q['tracks'][i]['name'], q['tracks'][i]['artists'][0]['name']) for i in range(len(q['tracks']))][:25]

[('Wish You Were Here', 'Pink Floyd'),
 ('Comfortably Numb', 'Pink Floyd'),
 ('My Sweet Lord - 2014 Mix', 'George Harrison'),
 ('Bohemian Rhapsody - 2011 Mix', 'Queen'),
 ('Walk On the Wild Side', 'Lou Reed'),
 ('American Idiot', 'Green Day'),
 ('Hey Jude - Remastered 2015', 'The Beatles'),
 ('My Generation - Stereo Version', 'The Who'),
 ('Another Brick in the Wall, Pt. 2', 'Pink Floyd'),
 ('Whole Lotta Love - 1990 Remaster', 'Led Zeppelin'),
 ('Float On', 'Modest Mouse'),
 ('Purple Haze', 'Jimi Hendrix'),
 ("Can't Stop", 'Red Hot Chili Peppers'),
 ('Last Nite', 'The Strokes'),
 ('Take It Easy - 2013 Remaster', 'Eagles'),
 ('A Horse with No Name', 'America'),
 ('Come Together - Remastered 2009', 'The Beatles'),
 ("Rocket Man (I Think It's Going To Be A Long, Long Time)", 'Elton John'),
 ('Down On The Corner', 'Creedence Clearwater Revival'),
 ('Kashmir - 1990 Remaster', 'Led Zeppelin'),
 ('Fortunate Son', 'Creedence Clearwater Revival'),
 ('Immigrant Song - 2012 Remaster', 'Led Zeppel

Well, it seems to be working.